In [1]:
import pandas as pd
import numpy as np 

In [2]:
n_rows = 10000
n_cols = 10000
a = np.random.randint(low=0, high=20, size=(n_rows,n_cols), dtype=np.int64)
df = pd.DataFrame(a)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,10,1,0,8,7,18,19,15,1,17,...,3,12,17,16,5,4,2,0,10,18
1,0,6,18,9,2,9,17,3,16,0,...,18,17,10,9,1,12,9,14,3,11
2,19,8,19,1,12,9,17,8,6,16,...,2,3,18,16,1,10,9,7,14,5
3,10,12,8,3,17,9,11,8,0,14,...,8,19,4,8,17,0,8,19,7,0
4,1,10,11,7,17,8,12,0,3,4,...,6,11,4,5,2,2,14,11,2,5


In [4]:
%%time

# Worse case scenario: double python for loop (O(n^2)) !!
sum = 0
for row_id in range(n_rows):
    for i in a[row_id,:]:
        sum = sum + i
sum

CPU times: user 8.59 s, sys: 22.6 ms, total: 8.61 s
Wall time: 8.62 s


950025215

In [5]:
%%time
a.sum()

CPU times: user 17.6 ms, sys: 693 µs, total: 18.3 ms
Wall time: 17.5 ms


950025215

In [6]:
%%time 
#column major sum
np.apply_along_axis(np.sum, 0, a).sum()

CPU times: user 254 ms, sys: 1.28 ms, total: 255 ms
Wall time: 255 ms


950025215

In [7]:
%%time
#row major sum
np.apply_along_axis(np.sum, 1, a).sum()

CPU times: user 42 ms, sys: 804 µs, total: 42.8 ms
Wall time: 42.2 ms


950025215

In [9]:
%%time
# Column-wise operation
a_col_major = np.ones((n_rows,n_cols), order='F')
np.apply_along_axis(np.sum, 0, a_col_major).sum()

CPU times: user 73.5 ms, sys: 65.9 ms, total: 139 ms
Wall time: 139 ms


100000000.0

In [10]:
%%time
# Row-wise operation
np.apply_along_axis(np.sum, 1, a_col_major).sum()

CPU times: user 262 ms, sys: 12.9 ms, total: 275 ms
Wall time: 274 ms


100000000.0

## Pandas

In [11]:
%%time
# Using pandas vectorized sum is much faster (here, it is called n_cols + 1 times)
df.sum().sum()

CPU times: user 42.3 ms, sys: 891 µs, total: 43.2 ms
Wall time: 42.6 ms


950025215

## Cython

In [12]:
%load_ext Cython

In [ ]:
%%cython -a

